# Treinamento de Modelo para Extração de Dados de Recibos Fiscais

Este notebook demonstra o fluxo de trabalho completo para preparar dados e treinar um modelo de reconhecimento de entidades nomeadas (NER) para extrair informações de recibos fiscais, como:
- Tipo de documento
- CNPJ
- Chave de acesso
- Data de emissão
- Valor total
- Número do documento
- Série

O processo envolve as seguintes etapas:
1. Instalação das dependências necessárias
2. Extração dos textos dos recibos do arquivo dataset.dart
3. Preparação e limpeza dos dados
4. Criação de anotações para NER
5. Treinamento de um modelo BERT pré-treinado para português
6. Avaliação do modelo
7. Uso do modelo para extrair informações de novos recibos

## 1. Instalação das Dependências

In [ ]:
# Instalar as dependências necessárias
!pip install numpy pandas scikit-learn torch transformers datasets matplotlib tqdm seqeval

## 2. Importações e Configurações Iniciais

In [ ]:
import os
import json
import re
import random
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorForTokenClassification
)
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from seqeval.metrics import classification_report, f1_score
from typing import List, Dict, Any, Tuple

# Configurações
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"  # Modelo pré-treinado em português
OUTPUT_DIR = "receipt_ner_model"
MAX_LENGTH = 512
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_EPOCHS = 10
SEED = 42

# Configurar as sementes para reprodutibilidade
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

## 3. Preparação do Dataset

### 3.1 Definição das Entidades e Constantes

In [ ]:
# Define as entidades que queremos extrair
ENTITIES = [
    "TIPO_DOCUMENTO",
    "CNPJ",
    "CHAVE_ACESSO",
    "DATA_EMISSAO",
    "VALOR_TOTAL",
    "NUMERO_DOCUMENTO",
    "SERIE"
]

# Defina as etiquetas para o modelo NER
LABELS = [
    "O",  # Outside (não é uma entidade)
    "B-TIPO_DOCUMENTO", "I-TIPO_DOCUMENTO",  # Tipo de documento (NFCe, NFe, SAT, CTe...)
    "B-CNPJ", "I-CNPJ",  # CNPJ
    "B-CHAVE_ACESSO", "I-CHAVE_ACESSO",  # Chave de acesso
    "B-DATA_EMISSAO", "I-DATA_EMISSAO",  # Data de emissão
    "B-VALOR_TOTAL", "I-VALOR_TOTAL",  # Valor total
    "B-NUMERO_DOCUMENTO", "I-NUMERO_DOCUMENTO",  # Número do documento
    "B-SERIE", "I-SERIE"  # Série do documento
]

# Mapeamento de ID para etiqueta e vice-versa
id2label = {i: label for i, label in enumerate(LABELS)}
label2id = {label: i for i, label in enumerate(LABELS)}

# Mapeamento dos códigos de modelo de documento para seus tipos
DOCUMENT_TYPE_MAP = {
    '01': 'NF',
    '02': 'NFVC',
    '04': 'NFP',
    '06': 'NFCE',
    '07': 'NFST',
    '08': 'CTRC',
    '09': 'CTAC',
    '10': 'CA',
    '11': 'CTFC',
    '13': 'BPR',
    '14': 'BPA',
    '15': 'BPNB',
    '16': 'BPF',
    '17': 'DT',
    '18': 'RMD',
    '20': 'OCC',
    '21': 'NFSC',
    '22': 'NFST',
    '23': 'GNRE',
    '24': 'AC',
    '25': 'MC',
    '26': 'CTMC',
    '27': 'NFTFC',
    '28': 'NFCG',
    '29': 'NFCA',
    '30': 'BRP',
    '2D': 'CFECF',
    '2E': 'BPEC',
    '55': 'NFE',
    '57': 'CTE',
    '59': 'CF',
    '60': 'CFEECF',
    '65': 'NFCE',
    '67': 'CTE',
    '8B': 'CTCA',
}

### 3.2 Extração dos Textos do Dataset

In [ ]:
def extract_texts_from_dart(dart_file_path: str) -> List[str]:
    """
    Extrai os textos do arquivo Dart.
    """
    with open(dart_file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Encontra todas as strings multi-linhas no arquivo Dart
    pattern = r"static const \w+ = r?'''(.*?)''';"
    matches = re.findall(pattern, content, re.DOTALL)
    
    texts = []
    for match in matches:
        # Normaliza o texto, removendo espaços extras e caracteres especiais
        text = match.strip()
        text = re.sub(r'\s+', ' ', text)
        texts.append(text)
    
    return texts

# Extrair os textos do arquivo Dart
dart_file_path = "dataset.dart"  # Altere para o caminho correto se necessário

try:
    texts = extract_texts_from_dart(dart_file_path)
    print(f"Extraídos {len(texts)} textos do arquivo Dart.")
    
    # Exibir uma amostra para verificação
    if texts:
        print("\nExemplo de texto extraído:")
        print("-" * 80)
        print(texts[0][:500] + "...")
        print("-" * 80)
except Exception as e:
    print(f"Erro ao extrair textos: {e}")
    
    # Criação de diretório para salvar os textos extraídos
    os.makedirs("dataset_prepared", exist_ok=True)
    
    # Salvar os textos brutos para uso posterior
    if texts:
        with open("dataset_prepared/raw_texts.txt", "w", encoding="utf-8") as f:
            for text in texts:
                f.write(text + "\n\n---SEPARATOR---\n\n")

### 3.3 Funções Auxiliares para Extração de Informações

In [ ]:
def is_valid_access_key(key: str) -> bool:
    """
    Verifica se a chave de acesso é válida com base nos prefixos conhecidos.
    """
    # Lista de prefixos válidos para chaves de acesso
    valid_prefixes = [
        "11", "12", "13", "14", "15", "16", "17", "21", "22", "23", "24", "25", 
        "26", "27", "28", "29", "31", "32", "33", "35", "41", "42", "43", "50", 
        "51", "52", "53"
    ]
    
    # Verifica se a chave tem 44 dígitos e começa com um prefixo válido
    if len(key) == 44 and key[:2] in valid_prefixes:
        return True
    return False

def extract_date_info_from_key(key: str) -> tuple:
    """
    Extrai o ano e mês da chave de acesso.
    
    Args:
        key: Chave de acesso de 44 dígitos
        
    Returns:
        Tupla contendo (ano, mês) da chave
    """
    if len(key) != 44:
        return None, None
    
    # Ano está nas posições 2-4 (2 dígitos)
    year = key[2:4]
    # Mês está nas posições 4-6 (2 dígitos)
    month = key[4:6]
    
    # Converte para números, adiciona 2000 ao ano para ter formato de 4 dígitos
    try:
        year = int(year)
        month = int(month)
        if 0 < month <= 12:  # Valida o mês
            # Formata para 4 dígitos (assumindo anos 2000)
            full_year = 2000 + year
            return str(full_year), f"{month:02d}"
    except ValueError:
        pass
    
    return None, None

def extract_document_type_from_key(key: str) -> str:
    """
    Extrai o tipo de documento a partir da chave de acesso.
    
    Args:
        key: Chave de acesso de 44 dígitos
        
    Returns:
        String com o tipo de documento ou string vazia se não for reconhecido
    """
    if len(key) != 44:
        return ""
    
    # O tipo de documento está nas posições 20-22
    model_code = key[20:22]
    
    # Retorna o tipo de documento correspondente ou string vazia se não encontrado
    return DOCUMENT_TYPE_MAP.get(model_code, "")

def clean_currency_value(value: str) -> str:
    """
    Limpa e padroniza um valor monetário.
    
    Args:
        value: Valor monetário como string (ex: "114,54", "R$ 50.00", etc.)
        
    Returns:
        Valor padronizado como string decimal com ponto (ex: "114.54")
    """
    if not value:
        return ""
    
    # Remove qualquer caractere que não seja dígito, vírgula ou ponto
    value = re.sub(r'[^\d,.]', '', value)
    
    # Substitui vírgula por ponto para padronização decimal
    value = value.replace(',', '.')
    
    # Se houver mais de um ponto, mantém apenas o último (caso de milhares)
    if value.count('.') > 1:
        parts = value.split('.')
        last_part = parts[-1]
        rest = ''.join(parts[:-1]).replace('.', '')
        value = f"{rest}.{last_part}"
    
    return value

### 3.4 Extração de Anotações dos Textos

In [ ]:
def create_annotations(texts: List[str]) -> List[Dict[str, Any]]:
    """
    Extrai as informações solicitadas dos textos e retorna um dataset estruturado
    no formato JSON.
    """
    annotations = []
    
    for i, text in enumerate(texts):
        # Inicializa o dicionário de dados com valores padrão
        data = {
            "tipo": "",
            "cnpj": "",
            "chave_acesso": "",
            "data_emissao": "",
            "valor_total_pago": "",
            "numero_documento": "",
            "serie": ""
        }
        
        # Extrai chave de acesso (44 dígitos) - PRIORIDADE ALTA
        chave_patterns = [
            r'(\d{4}\s\d{4}\s\d{4}\s\d{4}\s\d{4}\s\d{4}\s\d{4}\s\d{4}\s\d{4}\s\d{4}\s\d{4})',  # Com espaços
            r'(\d{44})'  # Sem espaços
        ]
        
        # Procura pela chave de acesso primeiro
        chave_acesso = ""
        for pattern in chave_patterns:
            chave_matches = re.findall(pattern, text)
            if chave_matches:
                for match in chave_matches:
                    # Remove todos os caracteres não numéricos
                    potential_key = re.sub(r'[^0-9]', '', match)
                    if len(potential_key) == 44 and is_valid_access_key(potential_key):
                        chave_acesso = potential_key
                        data["chave_acesso"] = chave_acesso
                        break
                if chave_acesso:
                    break
        
        # Variáveis para armazenar informações extraídas da chave
        key_year, key_month = None, None
        document_type_from_key = ""
        
        # Se encontrou a chave de acesso, extrai informações dela
        if chave_acesso:
            # Extrai CNPJ da chave de acesso (posições 6-20)
            data["cnpj"] = chave_acesso[6:20]
            
            # Extrai número do documento da chave de acesso (posições 31-37)
            data["numero_documento"] = str(int(chave_acesso[31:37]))  # Remove zeros à esquerda
            
            # Extrai ano e mês da chave de acesso
            key_year, key_month = extract_date_info_from_key(chave_acesso)
            
            # Extrai o tipo de documento da chave de acesso
            document_type_from_key = extract_document_type_from_key(chave_acesso)
            if document_type_from_key:
                data["tipo"] = document_type_from_key
        
        # Se não encontrou o tipo de documento na chave ou não encontrou a chave,
        # tenta identificar pelo texto (método original)
        if not document_type_from_key:
            tipo_documento = ""
            if "NFC-e" in text or "NOTA FISCAL DE CONSUMIDOR ELETRONICA" in text or "NFCe" in text:
                tipo_documento = "NFCE"
            elif "NF-e" in text or "NOTA FISCAL ELETRONICA" in text or "NFe" in text:
                tipo_documento = "NFE"
            elif "SAT" in text or "CUPOM FISCAL ELETRÔNICO - SAT" in text:
                tipo_documento = "SAT"
            elif "CT-e" in text or "CTE" in text or "CONHECIMENTO DE TRANSPORTE" in text:
                tipo_documento = "CTE"
            elif "CUPOM FISCAL" in text:
                tipo_documento = "CF"
            elif "DANFE" in text:
                tipo_documento = "NFE"
            data["tipo"] = tipo_documento
        
        # Se não encontrou a chave ou outras informações, continua com os métodos alternativos
        if not chave_acesso:
            # Tenta extrair CNPJ pelo método alternativo
            cnpj_matches = re.findall(r'CNPJ[:\s]*(\d{2}[\.]?\d{3}[\.]?\d{3}[/\.]?\d{4}[-\.]?\d{2}|\d{14})', text, re.IGNORECASE)
            if cnpj_matches:
                cnpj = cnpj_matches[0]
                # Remove todos os caracteres não numéricos
                cnpj = re.sub(r'[^0-9]', '', cnpj)
                data["cnpj"] = cnpj
            
            # Tenta extrair número do documento pelo método alternativo apenas se não foi extraído da chave
            if not data["numero_documento"]:
                numero_patterns = [
                    r'N[°º\.]?[:\s]*(?:0*)(\d+)',
                    r'Nº[:\s]*(?:0*)(\d+)',
                    r'N[°\.]?[:\s]*(?:0*)(\d+)',
                    r'n°[:\s]*(?:0*)(\d+)',
                    r'(?:NF(?:C|E)?-e|SAT)[:\s]*(?:n[°\.]?)?[:\s]*(?:0*)(\d+)',
                    r'Extrato\s+(?:N[°º\.]?)?[:\s]*(?:0*)(\d+)'
                ]
                
                for pattern in numero_patterns:
                    numero_matches = re.findall(pattern, text, re.IGNORECASE)
                    if numero_matches:
                        data["numero_documento"] = numero_matches[0]
                        break
        
        # Extrai data de emissão
        data_patterns = [
            r'Data\s+de\s+[Ee]miss[aã]o[:\s]*(\d{2}/\d{2}/\d{4})',
            r'[Ee]miss[ãa]o[:\s]*(\d{2}/\d{2}/\d{4})',
            r'DATA\s+DE\s+EMISSÃO[:\s]*(\d{2}/\d{2}/\d{4})',
            r'(\d{2}/\d{2}/\d{4})\s*-\s*\d{2}:\d{2}',  # Padrão data - hora
            r'(\d{2}/\d{2}/\d{4})'  # Qualquer data no formato DD/MM/AAAA
        ]
        
        # Armazena todas as datas encontradas para posterior validação
        all_dates = []
        for pattern in data_patterns:
            data_matches = re.findall(pattern, text, re.IGNORECASE)
            all_dates.extend(data_matches)
        
        # Se temos informações de data da chave de acesso, vamos usá-las para validar
        if key_year and key_month and all_dates:
            best_date = None
            for date_str in all_dates:
                # Extrair componentes da data encontrada (formato DD/MM/AAAA)
                day, month, year = date_str.split('/')
                
                # Verifica se o mês e o ano correspondem aos da chave de acesso
                if month == key_month and year == key_year:
                    best_date = date_str
                    break
            
            # Se encontrou uma data que corresponde à chave, usa essa
            if best_date:
                data["data_emissao"] = best_date
            elif all_dates:  # Caso contrário, usa a primeira data encontrada
                data["data_emissao"] = all_dates[0]
        elif all_dates:  # Se não há dados da chave, usa a primeira data encontrada
            data["data_emissao"] = all_dates[0]
        
        # Extrai valor total
        valor_patterns = [
            r'VALOR\s+TOTAL\s+(?:R\$)?\s*([\d\.]+,[\d]+)',
            r'VALOR\s+A\s+PAGAR\s+(?:R\$)?\s*([\d\.]+,[\d]+)',
            r'Total\s+(?:R\$)?\s*([\d\.]+,[\d]+)',
            r'TOTAL\s+(?:R\$)?\s*([\d\.]+,[\d]+)',
            r'Valor\s+Total:\s*(?:R\$)?\s*([\d\.]+,[\d]+)',
            r'R\$\s*([\d\.]+,[\d]+)',  # Padrão genérico de valores em reais
            r'([\d\.]+,[\d]+)'  # Qualquer valor numérico com decimal separado por vírgula
        ]
        
        for pattern in valor_patterns:
            valor_matches = re.findall(pattern, text, re.IGNORECASE)
            if valor_matches:
                # Limpa e padroniza o valor
                valor = clean_currency_value(valor_matches[0])
                data["valor_total_pago"] = valor
                break
        
        # Extrai série
        serie_patterns = [
            r'S[eé]rie[:\s]*([0-9]+)',
            r'SERIE[:\s]*([0-9]+)',
            r'SÉRIE[:\s]*([0-9]+)',
            r'Serie[:\s]*([0-9]+)'
        ]
        
        for pattern in serie_patterns:
            serie_matches = re.findall(pattern, text, re.IGNORECASE)
            if serie_matches:
                data["serie"] = serie_matches[0]
                break
        
        # Adiciona os dados extraídos à lista de anotações
        annotations.append({"id": i, "text": text, "annotations": data})
    
    return annotations

# Criar as anotações a partir dos textos extraídos
if 'texts' in locals() and texts:
    print("Gerando anotações a partir dos textos...")
    annotations = create_annotations(texts)
    print(f"Criadas {len(annotations)} anotações.")
    
    # Salvar anotações
    with open("dataset_prepared/annotations.json", "w", encoding="utf-8") as f:
        json.dump(annotations, f, ensure_ascii=False, indent=2)
    
    # Extrair os dados para um arquivo JSON separado
    extracted_data = []
    for ann in annotations:
        extracted_data.append({
            "id": ann["id"],
            **ann["annotations"]
        })
    
    with open("dataset_prepared/extracted_data.json", "w", encoding="utf-8") as f:
        json.dump(extracted_data, f, ensure_ascii=False, indent=2)
    
    print("Dados extraídos e salvos em dataset_prepared/")
else:
    print("Nenhum texto disponível para criar anotações.")

## 4. Preparação dos Dados para Treinamento

Vamos preparar o conjunto de dados para treinar o modelo de reconhecimento de entidades (NER).

In [ ]:
def convert_annotations_to_ner_format(annotations: List[Dict]) -> List[Dict]:
    """
    Converte as anotações no formato JSON para o formato NER adequado para treinamento.
    Cada token receberá uma tag usando o esquema BIO (Begin, Inside, Outside).
    
    Args:
        annotations: Lista de anotações obtidas do método create_annotations
        
    Returns:
        Lista de dicionários no formato {"tokens": [...], "ner_tags": [...], ...}
    """
    ner_data = []
    
    for ann in tqdm(annotations, desc="Convertendo para formato NER"):
        text = ann["text"]
        ann_data = ann["annotations"]
        
        # Tokenizar o texto (aqui, por simplicidade, dividimos por espaço)
        tokens = text.split()
        ner_tags = ["O"] * len(tokens)  # Inicialmente, todos os tokens são "Outside"
        
        # Tentar etiquetar cada entidade no texto
        for entity_type, entity_value in ann_data.items():
            if not entity_value:  # Se o valor da entidade estiver vazio, pular
                continue
                
            # Mapear os nomes de entidades no JSON para os nomes de entidades no NER
            entity_map = {
                "tipo": "TIPO_DOCUMENTO",
                "cnpj": "CNPJ",
                "chave_acesso": "CHAVE_ACESSO",
                "data_emissao": "DATA_EMISSAO",
                "valor_total_pago": "VALOR_TOTAL",
                "numero_documento": "NUMERO_DOCUMENTO",
                "serie": "SERIE"
            }
            
            if entity_type not in entity_map:
                continue
                
            ner_entity = entity_map[entity_type]
            
            # Procurar o valor da entidade no texto
            # Pode ser necessário ajustar essa lógica para entidades mais complexas
            value_tokens = entity_value.split()
            if not value_tokens:
                continue
                
            for i in range(len(tokens) - len(value_tokens) + 1):
                # Verificar se os tokens correspondem ao valor (ignorando case)
                if all(tokens[i+j].lower() == value_tokens[j].lower() for j in range(len(value_tokens))):
                    # Marcar o primeiro token como B (Begin)
                    ner_tags[i] = f"B-{ner_entity}"
                    # Marcar os tokens restantes como I (Inside)
                    for j in range(1, len(value_tokens)):
                        ner_tags[i+j] = f"I-{ner_entity}"
        
        # Adicionar o exemplo ao conjunto de dados NER
        ner_data.append({
            "id": ann["id"],
            "tokens": tokens,
            "ner_tags": ner_tags
        })
    
    return ner_data

# Carregar as anotações se não estiverem disponíveis
if 'annotations' not in locals() or not annotations:
    try:
        with open("dataset_prepared/annotations.json", "r", encoding="utf-8") as f:
            annotations = json.load(f)
        print(f"Carregadas {len(annotations)} anotações do arquivo.")
    except Exception as e:
        print(f"Erro ao carregar anotações: {e}")
        annotations = []

# Converter anotações para formato NER
if annotations:
    ner_data = convert_annotations_to_ner_format(annotations)
    print(f"Convertidos {len(ner_data)} exemplos para formato NER.")
    
    # Exibir um exemplo para verificação
    if ner_data:
        example = ner_data[0]
        print("\nExemplo de dados no formato NER:")
        print("-" * 80)
        for token, tag in zip(example["tokens"][:20], example["ner_tags"][:20]):
            print(f"{token} -> {tag}")
        print("...")
        print("-" * 80)
else:
    print("Nenhuma anotação disponível para conversão.")

## 5. Preparação do Dataset para Treinamento com Transformers

In [ ]:
def convert_to_datasets_format(ner_data):
    """
    Converte os dados NER para o formato esperado pela biblioteca Datasets.
    """
    # Dividir em conjuntos de treino, validação e teste
    train_data, test_data = train_test_split(ner_data, test_size=0.2, random_state=SEED)
    train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=SEED)  # 0.25 * 0.8 = 0.2
    
    # Converter para o formato do Datasets
    def convert_format(data):
        return {
            "tokens": [example["tokens"] for example in data],
            "ner_tags": [[label2id[tag] for tag in example["ner_tags"]] for example in data],
            "id": [example["id"] for example in data]
        }
    
    train_dataset = Dataset.from_dict(convert_format(train_data))
    val_dataset = Dataset.from_dict(convert_format(val_data))
    test_dataset = Dataset.from_dict(convert_format(test_data))
    
    # Criar um DatasetDict
    datasets = DatasetDict({
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset
    })
    
    return datasets

# Converter para formato de dataset
if 'ner_data' in locals() and ner_data:
    print("Convertendo para formato de dataset...")
    datasets = convert_to_datasets_format(ner_data)
    
    print(f"Criados datasets com:")
    print(f"  - {len(datasets['train'])} exemplos de treino")
    print(f"  - {len(datasets['validation'])} exemplos de validação")
    print(f"  - {len(datasets['test'])} exemplos de teste")
else:
    print("Nenhum dado NER disponível para conversão.")

## 6. Tokenização para o Modelo

In [ ]:
def tokenize_and_align_labels(examples):
    """
    Tokeniza os textos e alinha as etiquetas NER aos tokens produzidos pelo tokenizador.
    """
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=MAX_LENGTH,
        padding="max_length"
    )
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:
            # Tokens especiais ([CLS], [SEP], [PAD]) recebem -100 (ignorados na perda)
            if word_idx is None:
                label_ids.append(-100)
            # Para o primeiro token de uma palavra, use a etiqueta correspondente
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # Para subpalavras subsequentes, use -100 (ignoradas na perda)
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
            
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Carregar o tokenizador
print(f"Carregando tokenizador para o modelo {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenizar e alinhar as etiquetas
if 'datasets' in locals():
    print("Tokenizando datasets...")
    tokenized_datasets = datasets.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=datasets["train"].column_names
    )
    print("Datasets tokenizados e prontos para treinamento.")
else:
    print("Nenhum dataset disponível para tokenização.")

## 7. Definição do Modelo e Treinamento

In [ ]:
# Verificar disponibilidade de GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Configuração do modelo
if 'tokenized_datasets' in locals():
    print(f"Carregando modelo base {MODEL_NAME}...")
    model = AutoModelForTokenClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(LABELS),
        id2label=id2label,
        label2id=label2id
    )
    model.to(device)
    
    # Configuração de treinamento
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        evaluation_strategy="epoch",
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=NUM_EPOCHS,
        weight_decay=0.01,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        report_to="none",  # Desabilitar relatório para services externos
    )
    
    # Função de cálculo de métricas
    def compute_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)
        
        # Remover tokens ignorados (-100)
        true_predictions = [
            [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        
        # Calcular F1-score
        f1 = f1_score(true_labels, true_predictions)
        return {"f1": f1}
    
    # Colator de dados
    data_collator = DataCollatorForTokenClassification(tokenizer)
    
    # Criar o treinador
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )
    
    print("Modelo configurado e pronto para treinamento.")
    print("Você pode iniciar o treinamento executando a célula abaixo.")
else:
    print("Nenhum dataset tokenizado disponível para treinamento.")

## 8. Treinar o Modelo

Execute esta célula para iniciar o treinamento do modelo. Isso pode levar algum tempo, especialmente se você não tiver uma GPU disponível.

In [ ]:
# Treinar o modelo
if 'trainer' in locals():
    print("Iniciando treinamento...")
    trainer.train()
    print("Treinamento concluído!")
    
    # Salvar o modelo treinado
    trainer.save_model(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"Modelo salvo em {OUTPUT_DIR}")
else:
    print("Treinador não está disponível. Por favor, execute as células anteriores primeiro.")

## 9. Avaliar o Modelo no Conjunto de Teste

In [ ]:
# Avaliar o modelo no conjunto de teste
if 'trainer' in locals() and 'tokenized_datasets' in locals():
    print("Avaliando o modelo no conjunto de teste...")
    test_results = trainer.evaluate(tokenized_datasets["test"])
    print(f"Resultados da avaliação: {test_results}")
    
    # Obter previsões detalhadas
    predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
    predictions = np.argmax(predictions, axis=2)
    
    # Remover tokens ignorados (-100)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    # Imprimir relatório de classificação
    print("Relatório de classificação:")
    print(classification_report(true_labels, true_predictions))
else:
    print("Treinador ou datasets não disponíveis. Por favor, execute as células anteriores primeiro.")

## 10. Exemplo de Uso do Modelo Treinado para Extrair Informações de Novos Recibos

In [ ]:
def extract_info_from_receipt(text, model, tokenizer):
    """
    Extrai informações de um recibo fiscal usando o modelo treinado.
    
    Args:
        text: Texto do recibo
        model: Modelo NER treinado
        tokenizer: Tokenizador associado ao modelo
        
    Returns:
        Dicionário com as informações extraídas
    """
    # Tokenizar o texto
    tokens = text.split()
    
    # Tokenizar para o modelo
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH
    )
    
    # Mover para GPU se disponível
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        model = model.to("cuda")
    
    # Obter previsões
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Converter para labels
    predictions = outputs.logits.argmax(dim=2)
    word_ids = inputs.word_ids(batch_index=0)
    
    # Extrair entidades reconhecidas
    entities = {entity: "" for entity in ENTITIES}
    current_entity = None
    current_tokens = []
    
    for idx, (word_idx, pred) in enumerate(zip(word_ids, predictions[0])):
        if word_idx is None:
            continue
            
        label = id2label[pred.item()]
        
        if label.startswith("B-"):
            # Se estava coletando tokens para outra entidade, salvá-los
            if current_entity and current_tokens:
                entities[current_entity] = " ".join(current_tokens)
                
            # Iniciar nova entidade
            current_entity = label[2:]  # Remover o "B-"
            current_tokens = [tokens[word_idx]]
            
        elif label.startswith("I-") and current_entity == label[2:]:
            # Continuar adicionando tokens à entidade atual
            current_tokens.append(tokens[word_idx])
            
        elif label == "O" and current_entity:
            # Finalizar entidade atual
            entities[current_entity] = " ".join(current_tokens)
            current_entity = None
            current_tokens = []
    
    # Lidar com qualquer entidade restante no final
    if current_entity and current_tokens:
        entities[current_entity] = " ".join(current_tokens)
    
    # Retornar as entidades encontradas
    return entities

# Carregar o modelo salvo (se disponível)
try:
    if os.path.exists(OUTPUT_DIR):
        print(f"Carregando modelo salvo de {OUTPUT_DIR}...")
        model = AutoModelForTokenClassification.from_pretrained(OUTPUT_DIR)
        tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
        print("Modelo carregado com sucesso!")
        
        # Exemplo de uso com um texto de teste
        if 'texts' in locals() and texts:
            # Usar um texto do conjunto de dados que não foi usado no treinamento
            test_text = texts[-1]  # Usar o último texto como exemplo
            
            print("\nExtraindo informações do recibo de exemplo...")
            extracted_info = extract_info_from_receipt(test_text, model, tokenizer)
            
            print("\nInformações extraídas:")
            for entity, value in extracted_info.items():
                print(f"{entity}: {value}")
    else:
        print(f"Diretório do modelo {OUTPUT_DIR} não encontrado. Execute o treinamento primeiro.")
except Exception as e:
    print(f"Erro ao carregar o modelo: {e}")

## 11. Conclusão

Este notebook demonstrou todo o processo de preparação de dados e treinamento de um modelo de reconhecimento de entidades nomeadas (NER) para extração de informações de recibos fiscais. O fluxo de trabalho incluiu:

1. Extração dos textos do arquivo de dados
2. Preparação e limpeza dos dados
3. Criação de anotações para treinamento do modelo NER
4. Preparação dos datasets para treinamento
5. Treinamento do modelo utilizando um BERT pré-treinado para português
6. Avaliação do modelo no conjunto de teste
7. Uso do modelo para extrair informações de novos recibos

O modelo resultante pode ser usado para automatizar a extração de informações importantes de recibos fiscais, facilitando processos de gestão financeira, contabilidade e conformidade fiscal.